<a href="https://colab.research.google.com/github/Vinaypatil-Ev/vinEvPy-GoCoLab/blob/main/Tensorflow/TensorflowPrac20_writting_loop_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
def get_model():  
    inputs = tf.keras.Input(shape=(784, ))
    x = tf.keras.layers.Dense(64, activation="relu")(inputs)
    x = tf.keras.layers.Dense(128, activation="relu")(x)
    outputs = tf.keras.layers.Dense(10, activation="softmax")(x)
    return tf.keras.Model(inputs, outputs)

In [ ]:
(xtrn, ytrn), (xtst, ytst) = tf.keras.datasets.mnist.load_data()

In [ ]:
xtrn = xtrn.reshape((-1, 784))
xtst = xtst.reshape((-1, 784))

In [ ]:
xtrn.shape

(60000, 784)

In [ ]:
batch_size = 64
trn_data = tf.data.Dataset.from_tensor_slices((xtrn, ytrn))
trn_data = trn_data.shuffle(buffer_size=1024).batch(batch_size)
tst_data = tf.data.Dataset.from_tensor_slices((xtst, ytst))
tst_data = tst_data.batch(batch_size)

In [ ]:
optimizer = tf.keras.optimizers.SGD(learning_rate=1e-3)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

## Custom training loop

In [ ]:
epochs = 2
model = get_model()
for epoch in range(epochs):
    print(f"epoch{epoch}")
    for steps, (xtrn, ytrn) in enumerate(trn_data):
        with tf.GradientTape() as tape:
            ypred = model(xtrn, training=True)
            loss_value = loss_fn(ytrn, ypred)
        grad = tape.gradient(loss_value, model.trainable_weights)
        optimizer.apply_gradients(zip(grad, model.trainable_weights))

        if steps % 100 == 0:
            print(f"batch {steps}", "Loss:", float(loss_value))

        

epoch0
batch 0 Loss: 72.48542022705078
batch 100 Loss: 1.882691502571106
batch 200 Loss: 1.5710645914077759
batch 300 Loss: 1.1366631984710693
batch 400 Loss: 1.3355623483657837
batch 500 Loss: 1.4733967781066895
batch 600 Loss: 1.0884144306182861
batch 700 Loss: 0.8246899247169495
batch 800 Loss: 0.9663909673690796
batch 900 Loss: 0.21262231469154358
epoch1
batch 0 Loss: 0.6532958745956421
batch 100 Loss: 0.6560090780258179
batch 200 Loss: 0.4853571057319641
batch 300 Loss: 0.4727972149848938
batch 400 Loss: 0.6853070259094238
batch 500 Loss: 0.7546948194503784
batch 600 Loss: 0.714614987373352
batch 700 Loss: 0.3614707589149475
batch 800 Loss: 0.6650391817092896
batch 900 Loss: 0.25575166940689087


## Training loop with metrics

In [ ]:
model = get_model()
batch_size = 64
epochs = 4
optimizer = tf.keras.optimizers.SGD(learning_rate=1e-3)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
acc_metrics = tf.keras.metrics.SparseCategoricalAccuracy()
val_metrics = tf.keras.metrics.SparseCategoricalAccuracy()

In [ ]:
import time

In [ ]:
for epoch in range(epochs):
    print(f"epoch:{epoch}")
    start_time = time.time()
    for steps, (xtrn, ytrn) in enumerate(trn_data):
        with tf.GradientTape() as tape:
            pred = model(xtrn, training=True)
            loss_value = loss_fn(ytrn, pred)
        grad = tape.gradient(loss_value, model.trainable_weights)
        optimizer.apply_gradients(zip(grad, model.trainable_weights))
        acc_metrics.update_state(ytrn, pred)
        # if steps % 100 == 0:
            # print(f"steps:{steps}", "loss", loss_value) 
    trn_acc = acc_metrics.result()
    print("accuracy %.4f" % (float(trn_acc)))

    acc_metrics.reset_states()

    for xtst, ytst in tst_data:
        pred = model(xtst)
        val_metrics.update_state(ytst, pred)

    val_acc = val_metrics.result()
    print("val acc: %.4f" % (float(val_acc)))
    print("Time taken: %.2fs" % (float(time.time() - start_time)))


epoch:0
accuracy 0.7286
val acc: 0.8167
Time taken: 7.34s
epoch:1
accuracy 0.8497
val acc: 0.8449
Time taken: 7.38s
epoch:2
accuracy 0.8810
val acc: 0.8594
Time taken: 7.22s
epoch:3
accuracy 0.8984
val acc: 0.8687
Time taken: 7.32s


## speed up with tf.function()

In [ ]:
model = get_model()
batch_size = 64
epochs = 4
optimizer = tf.keras.optimizers.SGD(learning_rate=1e-3)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
acc_metrics = tf.keras.metrics.SparseCategoricalAccuracy()
val_metrics = tf.keras.metrics.SparseCategoricalAccuracy()
model = get_model()

In [ ]:
@tf.function
def train_step(x, y):
    with tf.GradientTape() as tape:
        pred = model(x, training=True)
        loss_value = loss_fn(ytrn, pred)
    grad = tape.gradient(loss_value, model.trainable_weights)
    optimizer.apply_gradients(zip(grad, model.trainable_weights))
    acc_metrics.update_state(ytrn, pred)

In [ ]:
@tf.function
def test_step(x, y):
    pred = model(x)
    val_metrics.update_state(y, pred)

In [ ]:
for epoch in range(epochs):
    print(f"epoch: {epoch}")
    start_time = time.time()
    for steps, (xtrn, ytrn) in enumerate(trn_data):
        train_step(xtrn, ytrn)
    trn_acc = acc_metrics.result()
    print("trn_acc: %.4f" % (trn_acc))
    for xtst, ytst in tst_data:
        test_step(xtst, ytst)
    val_acc = val_metrics.result()
    print("val_acc: %.4f" % (val_acc))   
    print("Time Taken: %.2fS" % (time.time() - start_time)) 

epoch: 0
trn_acc: 0.1632
val_acc: 0.0950
Time Taken: 2.28S
epoch: 1
trn_acc: 0.1670
val_acc: 0.0951
Time Taken: 1.81S
epoch: 2
trn_acc: 0.1684
val_acc: 0.0952
Time Taken: 1.70S
epoch: 3
trn_acc: 0.1691
val_acc: 0.0952
Time Taken: 1.68S


## End to End network with Gan Example

In [ ]:
descriminator = tf.keras.Sequential([
    tf.keras.Input((28, 28, 1)),
    tf.keras.layers.Conv2D(64, 3, (2, 2), padding="same"),
    tf.keras.layers.LeakyReLU(alpha=0.2),
    tf.keras.layers.Conv2D(128, 3, (2, 2), padding="same"),
    tf.keras.layers.LeakyReLU(alpha=0.2),
    tf.keras.layers.Dense(1),
], name="Descrimintor")

In [ ]:
latent_dim = 128
generator = tf.keras.Sequential([
    tf.keras.Input(shape=(latent_dim,)),
    tf.keras.layers.Dense(7 * 7 * 128),
    tf.keras.layers.LeakyReLU(alpha=0.2),
    tf.keras.layers.Reshape((7, 7, 128)),
    tf.keras.layers.Conv2DTranspose(128, 4, (2, 2), padding="same"),
    tf.keras.layers.LeakyReLU(alpha=0.2),
    tf.keras.layers.Conv2DTranspose(128, 4, (2, 2), padding="same"),
    tf.keras.layers.LeakyReLU(alpha=0.2),
    tf.keras.layers.Conv2D(1, (7, 7), padding="same", activation="sigmoid")
], name="Generator")

In [ ]:
d_optimizer = tf.keras.optimizers.Adam(learning_rate=0.003, name="d_opti")
g_optimizer = tf.keras.optimizers.Adam(learning_rate=0.003, name="g_opti")
loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=True, name="loss_fn")
batch_size = 64

In [ ]:
@tf.function
def train_step(real_img):
    random_latent_vector = tf.random.normal((batch_size, latent_dim), name="rlv1")
    gen_img = generator(random_latent_vector)
    combined_img = tf.concat([gen_img, real_img], axis=0)
    labels = tf.concat([
                        tf.ones((batch_size, 1)),
                        tf.zeros((real_img.shape[0], 1)),], axis=0)
    labels += 0.05 * tf.random.uniform(labels.shape, name="label_0")
    with tf.GradientTape() as tape:
        pred = descriminator(combined_img)
        d_loss_value = loss_fn(labels, pred)
    grad = tape.gradient(d_loss_value, descriminator.trainable_weights)
    d_optimizer.apply_gradients(zip(grad, descriminator.trainable_weights))

    random_latent_vector = tf.random.normal(shape=(batch_size, latent_dim), name="rlv2")
    false_labels = tf.zeros((batch_size, 1), name="false_label_0")

    with tf.GradientTape() as tape:
        pred = descriminator(generator(random_latent_vector))
        g_loss_value = loss_fn(false_labels, pred)
    grad = tape.gradient(g_loss_value, generator.trainable_weights)
    g_optimizer.apply_gradients(grad, generator.trainable_weights)

    return d_loss_value, g_loss_value, gen_img

In [ ]:
import os

In [ ]:
batch_size = 64

In [ ]:
(xtrn, _), (xtst, _) = tf.keras.datasets.mnist.load_data()
x = np.concatenate([xtrn, xtst])
x = x.astype("float32") / 255.0
x = x.reshape((-1, 28, 28, 1))
data = tf.data.Dataset.from_tensor_slices(x)
data = data.shuffle(buffer_size=1024).batch(batch_size)

In [ ]:
epochs = 1
save_dir = "./"

In [ ]:
for epoch in range(epochs):
    for steps, real_img in enumerate(data):
        d_loss, g_loss, gen_img = train_step(real_img)
        if steps % 200 == 0:
            print(f"d_loss {float(d_loss)}")
            print(f"g_loss {float(g_loss)} ")
            img = tf.keras.preprocessing.image.array_to_img(
                gen_img[0] * 255.0, scale=False
            )
            img.save(os.path.join(save_dir, "gen_img" + str(step) + ".png"))
        if step > 1:
            break

ValueError: ignored